In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
# 2025 年的測試資料為 'm05a_05F0287N_05F0055N_testingDataset'
test_file_path = r"D:\緯育課程\專題\TJR102_project\Data_M05A\m05a_05F0287N_05F0055N_testingDataset.csv"

# 載入 2025 年的測試資料
def load_data(test_file_path):
    df = pd.read_csv(test_file_path)
    # 確保資料是按照時間排序的
    # 如果您的資料沒有時間戳記或已確保排序，可以省略這一步
    # df = df.sort_values(by='時間戳記') # 假設有時間戳記列

    # 將特徵轉換為 PyTorch 張量
    # 使用 'Avg_speed' 和 'Total_volume' 為特徵
    # 並且Avg_speed是我們要預測的目標變數
    test_features = df[['Avg_speed', 'Total_volume']].values # 輸入特徵
    test_target = df['Avg_speed'].values # 預測目標

    # 將 numpy 陣列轉換為 PyTorch 張量
    test_features = torch.tensor(test_features, dtype=torch.float32)
    test_target = torch.tensor(test_target, dtype=torch.float32)
    return test_features, test_target

test_features, test_target = load_data(test_file_path)

# 使用與訓練集相同的 sequence_length 進行序列化  ####這邊還要從Model_build搬code過來
def create_sequences(test_features, test_target, sequence_length):
    xs, ys = [], []
    for i in range(len(test_features) - sequence_length):
        x = test_features[i:(i + sequence_length)]
        y = test_target[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return torch.stack(xs), torch.stack(ys)

sequence_length = 40 
X_test, y_test = create_sequences(test_features, test_target, sequence_length)

print(f"2025年測試集的輸入數據形狀: {X_test.shape}")
print(f"2025年測試集的目標數據形狀: {y_test.shape}")

2025年測試集的輸入數據形狀: torch.Size([14240, 40, 2])
2025年測試集的目標數據形狀: torch.Size([14240])


In [3]:
# 定義 LSTMModel class（確保結構與訓練時完全一致）
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 初始化隱藏狀態和細胞狀態
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTM 傳播
        out, _ = self.lstm(x, (h0, c0))

        # 取最後一個時間步的輸出
        out = self.fc(out[:, -1, :])
        return out

# 實例化模型，並確保超參數與訓練時一致
input_size = 2      
hidden_size = 33    # 使用最佳的 hidden_size################
num_layers = 3      # 使用最佳的 num_layers#################
output_size = 1     

loaded_model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 載入訓練好的權重
model_save_path = 'best_lstm_model.pth'  #若best_lstm_model.pth這個檔案不在當前資料夾，需放檔案絕對路徑
loaded_model.load_state_dict(torch.load(model_save_path))

# 將模型設定為評估模式
loaded_model.eval()
print("模型已成功載入並設定為評估模式。")

模型已成功載入並設定為評估模式。


In [4]:
from torch.utils.data import DataLoader, TensorDataset

# 將測試資料打包成 TensorDataset 和 DataLoader
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False) # 評估時通常不用 shuffle

test_predictions = []
test_actuals = []

# 禁用梯度計算以加速評估，並確保模型不會被修改
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        # 進行前向傳播
        outputs = loaded_model(batch_X)

        # 收集預測結果和真實值
        test_predictions.extend(outputs.squeeze().tolist())
        test_actuals.extend(batch_y.tolist())

# 將結果轉換為 numpy 陣列以便計算 MAPE
test_predictions = np.array(test_predictions)
test_actuals = np.array(test_actuals)

# 計算 MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    non_zero_indices = y_true != 0
    y_true_filtered = y_true[non_zero_indices]
    y_pred_filtered = y_pred[non_zero_indices]
    
    if len(y_true_filtered) == 0:
        return float('inf')

    return np.mean(np.abs((y_true_filtered - y_pred_filtered) / y_true_filtered)) * 100

mape_on_2025_data = mean_absolute_percentage_error(test_actuals, test_predictions)
print(f"模型在 2025 年測試集上的 MAPE: {mape_on_2025_data:.2f}%")

模型在 2025 年測試集上的 MAPE: 2.41%
